In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode


for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print "%s\t%d" % (word.lower(), 1)


Overwriting mapper.py


In [5]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0


def read_vocabulary(file_path):
    with open(file_path, 'r') as infile:
        return set(word.strip().lower() for word in infile.read())
stop_words = read_vocabulary("stop_words.txt")


for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
            if current_key in stop_words:
                print >> sys.stderr, "reporter:counter:Wiki stats,stop words,%d" % word_sum
            print >> sys.stderr, "reporter:counter:Wiki stats,total words,%d" % word_sum
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)
    if current_key in stop_words:
        print >> sys.stderr, "reporter:counter:Wiki stats,stop words,%d" % word_sum
    print >> sys.stderr, "reporter:counter:Wiki stats,total words,%d" % word_sum

Overwriting reducer.py


In [7]:
%%bash

NUM_REDUCERS=8

IN_DIR="/data/wiki/en_articles_part"
OUT_DIR="stopcount_result"$(date +"%s%6N")
OUT_LOG=out.$(date +"%s%6N").log

hdfs dfs -rm -r -f -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Streaming stop word count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt#stop_words.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input ${IN_DIR} \
    -output ${OUT_DIR} > /dev/null 2>${OUT_LOG}
#     -combiner "python reducer.py" \

cat ${OUT_LOG} >&2
grep ' words=' ${OUT_LOG} \
    | cut -d'=' -f2 \
    | paste -sd' ' \
    | awk '{ printf "%.2f", ($1 / $2 * 100) }'


2.62

17/10/15 17:27:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/10/15 17:27:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/10/15 17:27:08 INFO mapred.FileInputFormat: Total input files to process : 1
17/10/15 17:27:08 INFO mapreduce.JobSubmitter: number of splits:2
17/10/15 17:27:08 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
17/10/15 17:27:08 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1508087877012_0003
17/10/15 17:27:09 INFO impl.YarnClientImpl: Submitted application application_1508087877012_0003
17/10/15 17:27:09 INFO mapreduce.Job: The url to track the job: http://1c6f2be712dc:8088/proxy/application_1508087877012_0003/
17/10/15 17:27:09 INFO mapreduce.Job: Running job: job_1508087877012_0003
17/10/15 17:27:15 INFO mapreduce.Job: Job job_1508087877012_0003 running in uber mode : false
17/10/15 17:27:15 INFO mapreduce.Job:  map 0% reduce 0%
17/10/15 17:27:32 INFO m